# Simple Test Class


In [117]:
%%writefile testsfolder/my_tests.py

import unittest

class MyTests(unittest.TestCase):
    def test_something(self):
        self.assertEqual(1, 1)
    def test_something_else(self):
        self.assertEqual(0, 0)

Overwriting testsfolder/my_tests.py


# Simple Execution


## Specific File


In [118]:
!python3 -m unittest testsfolder/my_tests.py

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## Module

Note the following:

- no slashes (dots instead)
- no file extension


In [119]:
!python3 -m unittest testsfolder.my_tests

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## Specific Class

To do this, you have to load as a module instead of a file.


In [120]:
!python3 -m unittest testsfolder.my_tests.MyTests

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


## Specific Method

To do this, you have to:

- load as a module
- specify the classname as well


In [121]:
!python3 -m unittest testsfolder.my_tests.MyTests.test_something

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


# Printing from Within Test


In [122]:
%%writefile testsfolder/mytests_with_output.py

import unittest

class MyTests(unittest.TestCase):
    def test_something(self):
        x = 0
        
        print(f'ATTENTION: Value of x before test is {x}')
        self.assertEqual(1, 1)

Overwriting testsfolder/mytests_with_output.py


In [123]:
!python3 -m unittest testsfolder.mytests_with_output

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ATTENTION: Value of x before test is 0
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


# Test Class Members

You will often want common helper methods and variables to capture side effects in tests. It is safe to put these on the test class as long as they are **instance members** and not static members. Each test method gets executed **in its own class instance**, so as long as nothing is static, they won't interfere with each other.

Also note that to create instance variables, you need to **override \_\_init\_\_** correctly as shown.

Also note that the test framework uses the **test\_ prefix** to tell which methods are tests.


In [124]:
%%writefile testsfolder/mytests_with_members.py

import unittest

# Pretend this was imported from an API we will test.
class Transmitter:
    def transmit_x(self, fn):
        fn(10)
        
class MyTests(unittest.TestCase):
    def __init__(self, methodName='runTest'):
        super().__init__(methodName)
        
        # Instance variable
        self.captured_x = None
        
    # Helper method
    def capture_x(self, x):
        self.captured_x = x
        
    def test_1(self):
        transmitter = Transmitter()
        
        transmitter.transmit_x(self.capture_x)
        
        self.assertEqual(self.captured_x, 10)
        
    # If they ran in the same instance, this would fail.
    def test_2(self):
        self.assertIsNone(self.captured_x)

Overwriting testsfolder/mytests_with_members.py


In [125]:
!python3 -m unittest testsfolder.mytests_with_members

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


# Yoda Conditions

Some people prefer to write `self.assertEqual(10, x)` instead of `self.assertEqual(x, 10)`. For a lot of asserts like assertEqual, it doesn't matter. However, some asserts, especially the ones in **tensorflow.test.TestCase**, actually make less sense.

Overall, pay attention to the **param names** of assert methods to make sure you're using them as intended. For instance, if you used the non-yoda version for tensorflow.test.TestCase.assertAllEqual, the output would say "expected x but got 10" which would be confusing when debugging.


# Some Useful Asserts

There are a lot of asserts you can use in the TestCase class. Here are just a few examples.


In [126]:
%%writefile testsfolder/my_tests_asserts.py

import unittest

class MyTests(unittest.TestCase):
    def test_int(self):
        x = 0
        self.assertEqual(x, 0)
        
    def test_string(self):
        x = 'hi'
        self.assertEqual(x, 'hi')
        
    def test_none(self):
        x = None
        self.assertIsNone(x)
        
    def test_negation(self):
        self.assertIsNotNone(10)
        ## Use of Is is not consistent in the asserts.
        self.assertNotEqual(10, 11)
        
    def test_list(self):
        x = [1, 2, 3]
        self.assertListEqual(x, [1, 2, 3])
        
    def test_dict(self):
        x = {'a': 'hi'}
        self.assertDictEqual(x, {'a': 'hi'})
        
    def test_type(self):
        x = 5
        self.assertIsInstance(x, int)
        
    def test_reference(self):
        x = [1, 2, 3]
        self.assertIs(x, x)
        self.assertIsNot(x, [1, 2, 3])

Overwriting testsfolder/my_tests_asserts.py


In [127]:
!python3 -m unittest testsfolder.my_tests_asserts

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
........
----------------------------------------------------------------------
Ran 8 tests in 0.000s

OK


# Tensorflow

To assert between tensors and numpy arrays, python arrays, etc. you just need to derive from a **different base** to get the extra assert methods.

Unfortunately, you will also see the multitude of irrelevant warnings your TF prints if you haven't turned them off.


In [128]:
%%writefile testsfolder/my_tests_tf.py

import tensorflow as tf

class MyTests(tf.test.TestCase):
    def test_tensor(self):
        x = tf.constant([[1, 2, 3], [4, 5, 6]])
        # This is a case where you have to do Yoda conditions.
        self.assertAllEqual([[1, 2, 3], [4, 5, 6]], x)

Overwriting testsfolder/my_tests_tf.py


In [129]:
!python3 -m unittest testsfolder.my_tests_tf

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-05-19 11:45:44.668931: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 11:45:45.258768: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
s2023-05-19 11:45:46.019127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-19 11:45:46.044091: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have

# Mocking

The mocking library is very big and complex. Only a few examples are shown here.


## Mocking print()

Note below that prints that happen within the `with` block do not get printed to the console but instead get appended to our array. Once the block is over, printing goes to the console again.

WARNING: the reason you have to use builtins.print is because you can only mock something via its parent class or module. This applies to your own stuff as well.

WARNING: if you are testing a library that calls a function from a library, it won't get the mocked version unless it calls it via the parent (instead of importing directly as a name). This doesn't apply to builtins though.


In [130]:
%%writefile testsfolder/my_tests_mocking.py

import unittest
from unittest.mock import patch

def my_print(val):
    print(val)
    
class MyTests(unittest.TestCase):
    def test_my_print(self):
        printed = []
        with patch('builtins.print', new = lambda *args,**_: printed.append(args[0])):
            my_print(10)
            my_print('hi')
        print('ATTENTION: printed after mocking ended')
        self.assertListEqual(printed, [10, 'hi'])

Overwriting testsfolder/my_tests_mocking.py


In [131]:
!python3 -m unittest testsfolder.my_tests_mocking

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
ATTENTION: printed after mocking ended
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## Mocking Other Stuff


In [132]:
%%writefile testsfolder/my_tests_mocking_general.py

import unittest
from unittest.mock import patch

from os import path
    
class MyTests(unittest.TestCase):
    def test_dirname(self):
        captured = []
        # Need to use fully qualified name of symbol instead of how you imported it.
        with patch('os.path.dirname', new = lambda file: captured.append(file)):
            path.dirname('some file')
        # This one won't append.
        path.dirname(__file__)
        self.assertListEqual(captured, ['some file'])

Overwriting testsfolder/my_tests_mocking_general.py


In [133]:
!python3 -m unittest testsfolder.my_tests_mocking_general

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## Mocking Return Value

You don't have to do a lambda.


In [134]:
%%writefile testsfolder/my_tests_mocking_return.py

import unittest
from unittest.mock import patch

from os import path
    
class MyTests(unittest.TestCase):
    def test_dirname(self):
        with patch('os.path.dirname') as mock_dirname:
            mock_dirname.return_value = 'hi there'
            self.assertEqual(path.dirname('anything'), 'hi there')

Overwriting testsfolder/my_tests_mocking_return.py


In [135]:
!python3 -m unittest testsfolder.my_tests_mocking_return

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


## Fake

In this example, our class uses an expensive API that we do not want to actually use in our test. We can fake the API so that our class thinks it's using the real thing, and then test that our class did the right thing.


In [136]:
%%writefile testsfolder/my_tests_mocking_fake.py

import unittest
from unittest.mock import patch

# Assume this is in some API somewhere.
class SomeExpensiveApi:
    def __init__(self):
        self.x = 10
    def f(self):
        return self.x
    
def get_expensive_thing():
    return SomeExpensiveApi()

# Assume this is our own class we want to test.
class MyClass:
    def g(self):
        expensive = get_expensive_thing()
        return expensive.f()
    
class MyTests(unittest.TestCase):
    class MyFake:
        def __init__(self, val):
            self.val = val
        def f(self):
            return self.val
        
    def test_g(self):
        with patch(__name__ + '.get_expensive_thing', new = lambda: MyTests.MyFake(20)):
            m = MyClass()
            self.assertEqual(m.g(), 20)

Overwriting testsfolder/my_tests_mocking_fake.py


In [137]:
!python3 -m unittest testsfolder.my_tests_mocking_fake

/bin/bash: /home/davidpet/miniconda3/envs/ai/lib/libtinfo.so.6: no version information available (required by /bin/bash)
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


# pytest

`pytest` is a newer alternative to `unittest` with the following high-level differences:

1. must be `pip` installed
1. not class-based by default (though you can use classes)
1. autodiscovery by filenames with `test_*.py` and `*_test.py`
1. uses built-in `assert` statements instead of assert helper methods of a base class
1. run as a __comand__ instead of module (no `-m`)

## Basic Execution

1. `pytest` command will __recursively autodiscover__ within current folder:
   - all files that looks like `*_test.py` or `test_*.py`
   - all functions that starts with `test_`
     - plus other kinds of tests that will be shown later
   - you can pass in other options to change where and how it looks (eg. no recursion)
1. `assert` conditions used (with or without messages) to pass/fail tests
   - first failure fails the whole test
   - no `()` because `assert` is special python syntax
1. `-k {keyword}` used to only execute files matching the keyword in the name
   - more on this later, but needed here to target specific tests
1. `assert` is not required and is not the return value
   - no assert = test __always passes__

In [30]:
%%writefile testsfolder/pytest1_test.py

def test_sample():
    assert 0 == 0
    assert 1 == 0, 'this is my failure message'
    assert 0 == 0, "this won't run"
    
def test_sample_passes():
    assert 0 == 0
    
def test_nothing():
    pass

Overwriting testsfolder/pytest1_test.py


In [124]:
!cd testsfolder
!pytest -k pytest1

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 27 items / 23 deselected / 4 selected                                

testsfolder/pytest10_test.py .                                           [ 25%]
testsfolder/pytest1_test.py F..                                          [100%]

=================================== FAILURES ===================================
_________________________________ test_sample __________________________________

    def test_sample():
        assert 0 == 0
>       assert 1 == 0, 'this is my failure message'
E       AssertionError: this is my failure message
E       assert 1 == 0

testsfolder/pytest1_test.py:4: AssertionError
=========================== short test summary info ============================
FAILED testsfolder/pytest1_test.py::test_sample - AssertionError: this is my fail

## Asserting

Assert is specialized by `pytest` to smartly display some information such as operators used, etc. instead of just showing the values.

Returning a value other than `None` will generate a warning (as below) because (as shown below) it is a source of confusion where you could have a test __pass wrongly__.

In [21]:
%%writefile testsfolder/pytest2_test.py

def f(x):
    return x**2

def test_sample():
    assert f(2) == 4**2 + 1
    
def test_bad_return():
    return f(2) == 5  # should fail, but doesn't

Overwriting testsfolder/pytest2_test.py


In [22]:
!cd testsfolder
!pytest -k pytest2

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 4 items / 2 deselected / 2 selected                                  

testsfolder/pytest2_test.py F.                                           [100%]

=================================== FAILURES ===================================
_________________________________ test_sample __________________________________

    def test_sample():
>       assert f(2) == 4**2 + 1
E       assert 4 == ((4 ** 2) + 1)
E        +  where 4 = f(2)

testsfolder/pytest2_test.py:6: AssertionError
=============================== warnings summary ===============================
testsfolder/pytest2_test.py::test_bad_return
  /Users/davidpetrofsky/miniforge3/envs/ai/lib/python3.10/site-packages/_pytest/python.py:161: PytestReturnNotNoneWarning: Test functions should return None, but testsfo

## Testing for Exceptions

use `pytest.raises()` (which is a __context manager__) to require an exception to be thrown by a code block and optionally capture that exception to do further tests after.

There are options you can pass into `pytest.raises()` such as matching exception messages.

In [28]:
%%writefile testsfolder/pytest3_test.py

import pytest

def f():
    raise ZeroDivisionError()
    
def g():
    return None
    
def test_exceptions_pass():
    with pytest.raises(ZeroDivisionError):
        f()

def test_exceptions_fail():
    with pytest.raises(ZeroDivisionError):
        g()
        
def test_exceptions_capture():
    with pytest.raises(ZeroDivisionError) as exc:
        f()
    print('Captuerd exception:', exc) # this won't actually print - just here for demonstration

Overwriting testsfolder/pytest3_test.py


In [29]:
!cd testsfolder
!pytest -k pytest3

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 7 items / 4 deselected / 3 selected                                  

testsfolder/pytest3_test.py .F.                                          [100%]

=================================== FAILURES ===================================
_____________________________ test_exceptions_fail _____________________________

    def test_exceptions_fail():
>       with pytest.raises(ZeroDivisionError):
E       Failed: DID NOT RAISE <class 'ZeroDivisionError'>

testsfolder/pytest3_test.py:15: Failed
=========================== short test summary info ============================
FAILED testsfolder/pytest3_test.py::test_exceptions_fail - Failed: DID NOT RAISE <class 'ZeroDivisionError'>
================== 1 failed, 2 passed, 4 deselected in 0.04s ===================


## Printing from Tests

`stdout` and `stderr` are __captured__ by tests by default and will __not display__ unless the test fails.

If you want to see all output as it happens, pass the `-s` switch (can think of as "show") at the terminal.

In [36]:
%%writefile testsfolder/pytest4_test.py

import sys

def test_printing():
    print('hi')
    print('bye', file=sys.stderr)
    
def test_printing_fail():
    print('hi')
    print('bye', file=sys.stderr)
    assert False

Overwriting testsfolder/pytest4_test.py


In [37]:
!cd testsfolder
!pytest -k pytest4

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 10 items / 8 deselected / 2 selected                                 

testsfolder/pytest4_test.py .F                                           [100%]

=================================== FAILURES ===================================
______________________________ test_printing_fail ______________________________

    def test_printing_fail():
        print('hi')
        print('bye', file=sys.stderr)
>       assert False
E       assert False

testsfolder/pytest4_test.py:11: AssertionError
----------------------------- Captured stdout call -----------------------------
hi
----------------------------- Captured stderr call -----------------------------
bye
=========================== short test summary info ============================
FAILED testsfolder/pytest4_test

In [38]:
!cd testsfolder
!pytest -s -k pytest4

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 10 items / 8 deselected / 2 selected                                 

testsfolder/pytest4_test.py hi
bye
.hi
bye
F

=================================== FAILURES ===================================
______________________________ test_printing_fail ______________________________

    def test_printing_fail():
        print('hi')
        print('bye', file=sys.stderr)
>       assert False
E       assert False

testsfolder/pytest4_test.py:11: AssertionError
=========================== short test summary info ============================
FAILED testsfolder/pytest4_test.py::test_printing_fail - assert False
================== 1 failed, 1 passed, 8 deselected in 0.04s ===================


## Classes

1. Classes that start with `Test`, and have __no constructor__, are __instantiated__ to run their test methods.  They can be run from the same file as bare methods as shown below.

1. Test methods in a class can be __static, class, or instance methods__, but they must start with `test_` (and not the other way around).

1. All other methods and classes are ignored as tests, but can be used as __helpers__ as you wish.

1. State can __leak between tests__ in the following ways (all bad practices for testing because you have to rely on speicifc test order, etc.):
   - global variables
   - static class fields
   - instance fields
   
1. `unittest.TestCase` subclasses are also picked up as tests, __regardless of class name__.
   - so that you can keep using your old unit tests
     - as long as the __filenames are right__

In [74]:
%%writefile testsfolder/pytest5_test.py

count = 0

class TestMyStuff:
    class_count = 0
    
    def test_sample1(self):
        global count
    
        print('count from test_sample1:', count)
        count = count + 1
        
        print('class_count from test_sample1:', self.class_count)
        self.class_count = self.class_count + 1
        
        pass
    
    @staticmethod
    def test_sample2():
        global count
    
        print('count from test_sample2:', count)
        count = count + 1
        
        print('class_count from test_sample2:', TestMyStuff.class_count)
        TestMyStuff.class_count = TestMyStuff.class_count + 1
    
        pass
    
    def test_sample4(self):
        print('class_count from test_sample4:', self.class_count)
        pass
    
    def some_method(self):
        pass # ignored because of method name
    
    def some_test(self): # ignored because of method name
        pass
    
class OtherClass: # ignored because of class name
    def test_something(self):
        pass

class OtherTest: # ignored because of class name
    def test_something(self):
        pass

def test_sample3():
    global count
    
    print('count from test_sample3:', count)
    count = count + 1
    pass

Overwriting testsfolder/pytest5_test.py


In [75]:
!cd testsfolder
!pytest -s -k pytest5

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 14 items / 10 deselected / 4 selected                                

testsfolder/pytest5_test.py count from test_sample1: 0
class_count from test_sample1: 0
.count from test_sample2: 1
class_count from test_sample2: 0
.class_count from test_sample4: 1
.count from test_sample3: 2
.

======================= 4 passed, 10 deselected in 0.01s =======================


In [82]:
%%writefile testsfolder/pytest6_test.py

import unittest

class MyStuff(unittest.TestCase):
    def test_something(self):
        self.assertTrue(1 == 0)

Overwriting testsfolder/pytest6_test.py


In [84]:
!cd testsfolder
!pytest -k pytest6

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 15 items / 14 deselected / 1 selected                                

testsfolder/pytest6_test.py F                                            [100%]

=================================== FAILURES ===================================
____________________________ MyStuff.test_something ____________________________

self = <pytest6_test.MyStuff testMethod=test_something>

    def test_something(self):
>       self.assertTrue(1 == 0)
E       AssertionError: False is not true

testsfolder/pytest6_test.py:6: AssertionError
=========================== short test summary info ============================
FAILED testsfolder/pytest6_test.py::MyStuff::test_something - AssertionError: False is not true
======================= 1 failed, 14 deselected in 0.04s =================

## Fixtures

Fixtures provide a sophisticated and flexible way to provide __common test data/config__ and __shared state__ as needed for tests.

It is meant to cover the `Arrange` part of `Arrange/Act/Assert`, but it can sometimes cover the `Act` part too.

The concept of __setup/teardown__ that is usually done by methods in other frameworks is also handled more naturally within the fixture system instead.

### Basic Usage

In the simple case, a fixture is a __function that acts like a variable__.  When the fixture is requested, it is __memoized__ for the scope for which it is configured (__function/test__ by default).

Fixtures depend on other fixtures by using __parameters of the same name__ to signal to `pytest` that it should call the function and inject the value.  Tests depend on fixtures the same way.

In the example below:

1. `my_fruit` is requested (called) explicitly by tests 1 and 3 and __implicitly__ by test 2.
1. `my_fruit` is only called once by test 3 due to __memoization__
   - `my_basket` uses the cached result
   - __multiple fixtures__ can rely on the same (__possibly expensive__) base fixture without a performance hit this way
1. Fixtures can be requested in __any order__, but __no duplicates__ in the args list explicitly
   - you can predict the order of actual function calls by thinking of how the calls and memoization work

In [120]:
%%writefile testsfolder/pytest7_test.py

import pytest

@pytest.fixture
def my_fruit():
    print('my_fruit requested')
    return "apple"

@pytest.fixture
def my_basket(my_fruit):
    print("my_basket requested")
    return ["orange", "banana", my_fruit]

def test_1(my_fruit):
    assert my_fruit == 'apple'
    
def test_2(my_basket):
    assert my_basket[2] == 'apple'
    
def test_3(my_fruit, my_basket):
    assert my_fruit in my_basket
    
def test_4(my_basket, my_fruit):
    assert my_fruit in my_basket

Overwriting testsfolder/pytest7_test.py


In [121]:
!cd testsfolder
!pytest -k pytest7 -s

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 26 items / 22 deselected / 4 selected                                

testsfolder/pytest7_test.py my_fruit requested
.my_fruit requested
my_basket requested
.my_fruit requested
my_basket requested
.my_fruit requested
my_basket requested
.

======================= 4 passed, 22 deselected in 0.02s =======================


### Mutations

If a fixture returns a __mutable object__, and it is mutated in either a __test or fixture__, the mutation will stick for the __rest of the scope__ (by default the test/function).

This can be used as a way to inject variations of test data by controlling the __order of fixture parameters__ in a test.  This could also encroach on the `Act` part of `Arrange/Act/Assert` if you use a fixture to actually do business logic on data.

In [91]:
%%writefile testsfolder/pytest8_test.py

import pytest

@pytest.fixture
def my_fruit():
    return "apple"

@pytest.fixture
def my_basket(my_fruit):
    return ["orange", "banana", my_fruit]

@pytest.fixture
def stuff_basket(my_basket):
    my_basket.append("watermelon")  # didn't bother to return (None) because not going to read it

def test_1(my_basket):
    assert "watermelon" not in my_basket
    
def test_2(my_basket, stuff_basket):
    assert "watermelon" in my_basket # mutated by the fixture
    
def test_3(my_basket):
    assert "watermelon" not in my_basket  # no longer mutated because scope reset

Overwriting testsfolder/pytest8_test.py


In [92]:
!cd testsfolder
!pytest -k pytest8

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 21 items / 18 deselected / 3 selected                                

testsfolder/pytest8_test.py ...                                          [100%]

======================= 3 passed, 18 deselected in 0.01s =======================


### Fixtures in Classes

Like tests, fixtures can be __instance, static, or class methods__ of a class. They are still used the same way, from within the class.

`pytest` automatically figures out where to get a fixture based on the __param name__ alone, within the __lexical scope of the test/fixture__.  Note that this means you can __shadow__ a fixture of the same name from outside a lexical scope.

Because we have not changed any scopes, these fixtures are still function-scoped, and thus this example __behaves the same way as above__.

In [114]:
%%writefile testsfolder/pytest9_test.py

import pytest

@pytest.fixture
def my_fruit():
    print('my_fruit requested')
    return "apple"

@pytest.fixture
def my_fruit2():  # shadowed within the class because of duplicate name
    print('my_fruit2 outside requested')
    return "apple"

class TestMyStuff:
    @pytest.fixture
    def my_fruit2(self):
        print('my_fruit2 inside requested')
        return "apple"

    @pytest.fixture
    def my_basket(self, my_fruit, my_fruit2):
        print("my_basket requested")
        return ["orange", "banana", my_fruit, my_fruit2]

    @pytest.fixture
    @staticmethod
    def stuff_basket(my_basket):
        my_basket.append("watermelon")

    def test_1(self, my_fruit):
        assert my_fruit == 'apple'

    def test_2(self, my_basket):
        assert my_basket[2] == 'apple'

    def test_3(self, my_fruit, my_basket):
        assert my_fruit in my_basket
        
    def test_4(self, my_basket, stuff_basket):
        assert "watermelon" in my_basket

Overwriting testsfolder/pytest9_test.py


In [115]:
!cd testsfolder
!pytest -s -k pytest9

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 25 items / 21 deselected / 4 selected                                

testsfolder/pytest9_test.py my_fruit requested
.my_fruit requested
my_fruit2 inside requested
my_basket requested
.my_fruit requested
my_fruit2 inside requested
my_basket requested
.my_fruit requested
my_fruit2 inside requested
my_basket requested
.

======================= 4 passed, 21 deselected in 0.02s =======================


### Fixture as Factory Pattern

You can __return a function__ in order to inject a __callable object__ as a fixture.

In [122]:
%%writefile testsfolder/pytest10_test.py

import pytest

class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

@pytest.fixture
def make_person():
    def _make(name='bob',age=30):
        return Person(name, age)
    return _make

def test_person(make_person):
    person = make_person()
    assert person.name == 'bob'

Writing testsfolder/pytest10_test.py


In [123]:
!cd testsfolder
!pytest -k pytest10

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 27 items / 26 deselected / 1 selected                                

testsfolder/pytest10_test.py .                                           [100%]

======================= 1 passed, 26 deselected in 0.02s =======================


### autouse

Adding the boolean `autouse` parameter to the `fixture` decorator means the fixture will __always be implicitly requested__ within its __lexical scope__ (not its configured scope, which we'll cover next).

You can also include it __explicitly__ if you need to, which will just use the memoized autoused one.

This is a way to do __mutations on test data__ within certain test classes, as shown in the second (more concrete) example.

In [136]:
%%writefile testsfolder/pytest11_test.py

import pytest

@pytest.fixture(autouse=True)
def my_fruit():
    print('my_fruit requested')
    return 'apple'

@pytest.fixture(autouse=True)
def my_fruit2():
    print('my_fruit2 requested')
    return 'banana'

class TestMyStuff:
    @pytest.fixture(autouse=True)
    def my_basket(self, my_fruit):
        print('my_basket requested')
        return [my_fruit]
    
    #fruit, fruit2, and basket requested 3 times total even though not explicitly used here
    def test_autouse(self):
        pass
    
    def test_autouse2(self):
        pass
    
    def test_autouse3(self, my_fruit2, my_basket, my_fruit): # all cached before seen here
        pass
    
# only fruit and fruit2 requested
def test_autouser4():
    pass

Overwriting testsfolder/pytest11_test.py


In [137]:
!cd testsfolder
!pytest -s -k pytest11

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 31 items / 27 deselected / 4 selected                                

testsfolder/pytest11_test.py my_fruit requested
my_fruit2 requested
my_basket requested
.my_fruit requested
my_fruit2 requested
my_basket requested
.my_fruit requested
my_fruit2 requested
my_basket requested
.my_fruit requested
my_fruit2 requested
.

======================= 4 passed, 27 deselected in 0.02s =======================


In [142]:
%%writefile testsfolder/pytest12_test.py

import pytest

@pytest.fixture(autouse=True)
def my_fruit():
    return 'apple'

@pytest.fixture(autouse=True)
def my_fruit2():
    return 'banana'

@pytest.fixture(autouse=True)
def my_basket():
    return []

class TestEmptyBasket:
    def test_length(self, my_basket):
        assert len(my_basket) == 0
        
class TestSingleItemBasket:
    @pytest.fixture(autouse=True)
    def add_fruit_to_basket(self, my_basket, my_fruit):
        my_basket.append(my_fruit)
        
    def test_length(self, my_basket):
        assert len(my_basket) == 1
        
    def test_membership(self, my_basket, my_fruit):
        assert my_fruit in my_basket

Overwriting testsfolder/pytest12_test.py


In [143]:
!cd testsfolder
!pytest -k pytest12

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 34 items / 31 deselected / 3 selected                                

testsfolder/pytest12_test.py ...                                         [100%]

======================= 3 passed, 31 deselected in 0.02s =======================


### scope

Fixtures are scoped to `function` __by default__, which means they are re-called and cached for each test function/method. Other possible scopes (which you provide in the `fixture()` decorator) are: `class`, `module`, `package`, and `session`. You can also pass a callable to determine it __dynamically__.

The scope of a fixture is __separate__ from its __lexical scope__ (where it lives), as shown below. `autouse` will control at what level the fixture is automatically requested, but `scope` will determine how long the cached value is reused.

An example of how to use this would be to have a __class__ with some __common setup__ and then some tests that will `assert` different things on that setup.

__NOTE__: for the purpose of fixture scope, a __top-level function__ is treated like its __own class__.

In [156]:
%%writefile testsfolder/pytest13_test.py

import pytest

@pytest.fixture(scope='class')
def my_fruit():
    print('my_fruit requested')
    return 'apple'

class TestMyStuff:
    @pytest.fixture(scope='class')
    def my_basket(self, my_fruit):
        print('my_basket requested')
        return [my_fruit]
    
    @pytest.fixture(scope='class', autouse=True)
    def my_auto(self):
        print('my_auto requested')  # only requested once for the class
    
    def test_1(self, my_fruit, my_basket):
        pass # shared my_fruit with test_2
    
    def test_2(self, my_fruit, my_basket):
        pass  # shared my_fruit with test_1
    
def test_3(my_fruit):
    pass   # separate my_fruit

def test_4(my_fruit):
    pass   # separate my_fruit

Overwriting testsfolder/pytest13_test.py


In [164]:
!cd testsfolder
!pytest -s -k pytest13

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 39 items / 35 deselected / 4 selected                                

testsfolder/pytest13_test.py my_auto requested
my_fruit requested
my_basket requested
..my_fruit requested
.my_fruit requested
.

======================= 4 passed, 35 deselected in 0.02s =======================


In [165]:
%%writefile testsfolder/pytest14_test.py

import pytest

class Test3Fruits:
    @pytest.fixture(scope='class')
    def my_fruit(self):
        return 'apple'
    
    @pytest.fixture(scope='class')
    def my_basket(self, my_fruit):
        return [my_fruit]
    
    @pytest.fixture(scope='class', autouse=True)
    def stuff_basket(self, my_fruit, my_basket):
        my_basket.append(my_fruit)
        my_basket.append(my_fruit)
    
    def test_length(self, my_basket):
        assert len(my_basket) == 3
        
    def test_membership(self, my_fruit, my_basket):
        assert my_fruit in my_basket

Overwriting testsfolder/pytest14_test.py


In [166]:
!cd testsfolder
!pytest -k pytest14

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 40 items / 38 deselected / 2 selected                                

testsfolder/pytest14_test.py ..                                          [100%]

======================= 2 passed, 38 deselected in 0.02s =======================


### Setup/Teardown

__Setup__ is already a part of how fixtures are structured - you define scopes and dependencies within lexical scopes like classes to decide what is available in tests and how it is shared between tests.

Your fixture might have a side effect on the system (eg. create a temp folder, open a network connection, etc.) that needs to be __cleaned up__ when it goes out of `scope`. This is supported in the fixture system this way:

1. `yield` instead of `return` in the fixture
1. do cleanup code after that `yield`

Cleanup code will be called in __reverse stack order__ as you'd intuitively expect so that things can still see their dependencies.

__NOTE__: to be safe from errors between fixtures, make each fixture a __single modifiable piece of state__.

In [175]:
%%writefile testsfolder/pytest15_test.py

import pytest

@pytest.fixture(scope='class')
def my_fruit():
    yield 'apple'
    print('cleaning up apple')  # cleaned up twice (end of class and end of outside function)

class TestMyStuff:
    @pytest.fixture(scope='class')
    def my_basket(self, my_fruit):
        yield [my_fruit]
        print('cleaning up basket')  # only cleaned up once (end of class)
        
    def test_1(self, my_basket):
        pass
    
    def test_2(self, my_basket):
        pass
    
def test_3(my_fruit):
    pass

Overwriting testsfolder/pytest15_test.py


In [176]:
!cd testsfolder
!pytest -s -k pytest15

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 43 items / 40 deselected / 3 selected                                

testsfolder/pytest15_test.py ..cleaning up basket
cleaning up apple
.cleaning up apple


======================= 3 passed, 40 deselected in 0.02s =======================


### Old-style 'setup' function pattern

Just by using the mechanics of how scopes and autouse work, you can make a `setup()` function for a class that could be scoped to each test or the whole class, and provide teardown code within it.

In [181]:
%%writefile testsfolder/pytest16_test.py

import pytest

class TestMyStuff:
    @pytest.fixture(scope='function', autouse=True)
    def setup(self):
        print('setup called')
        yield
        print('teardown called')
        
    def test_1(self):
        pass
    
    def test_2(self):
        pass

Overwriting testsfolder/pytest16_test.py


In [182]:
!cd testsfolder
!pytest -s -k pytest16

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 45 items / 43 deselected / 2 selected                                

testsfolder/pytest16_test.py setup called
.teardown called
setup called
.teardown called


======================= 2 passed, 43 deselected in 0.02s =======================


### Fixtures from Libraries

In order to use fixtures across files, you just need to make sure the __fixture name__ is __lexically available__ in the context where it's injected as a parameter.

This is the key to making shared test code and also to accessing __built-in fixtures__ (see next).

In [183]:
%%writefile testsfolder/pytest17_lib.py

import pytest

@pytest.fixture
def my_fruit():
    return 'apple'

Writing testsfolder/pytest17_lib.py


In [192]:
%%writefile testsfolder/pytest18_test.py

import pytest
import testsfolder.pytest17_lib as thelib
from testsfolder.pytest17_lib import my_fruit

@pytest.fixture
def my_basket(my_fruit):
    return [my_fruit]

def test_import(my_fruit, my_basket):
    assert my_fruit == 'apple'
    assert my_fruit in my_basket
    assert len(my_basket) == 1

Overwriting testsfolder/pytest18_test.py


In [193]:
!cd testsfolder
!pytest -k pytest18

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 46 items / 45 deselected / 1 selected                                

testsfolder/pytest18_test.py .                                           [100%]

======================= 1 passed, 45 deselected in 0.02s =======================


### Built-In Fixtures

Just like how you can share your own functions in a library and use them as fixtures by using the param name matching the imported function name, the `pytest` module has several built-in fixtures that you can import and use by name.

These fixtures can be injected into __your fixtures and tests__.  However, they are __not directly importable__.  You just use the __correct param names__ and `pytest` understands what you want.

This example shows the most commonly used built-in fixture.  The `request` fixture is used to get information about the __specific test__ from within the test or within a fixture called by the test.

For a full list: http://docs.pytest.org/en/stable/reference/fixtures.html#fixture

In [208]:
%%writefile testsfolder/pytest19_test.py

import pytest

@pytest.fixture
def my_fruit(request):  # we didn't import 'request' - pytest just knows what it is
    print('requested apple')
    print(request.node)   # test_request or TestMyStuff.test_request_in_class
    print(request.node.cls)  # None or TestMyStuff
    return "apple"

def test_request(request, my_fruit):
    print(request.node)  # test_request (will match in fixture)
    assert my_fruit == 'apple'
    
class TestMyStuff:
    def test_request_in_class(self, my_fruit, request):
        print(request.node) # TestMyStuff.test_request_in_class (will match in fixture)

Overwriting testsfolder/pytest19_test.py


In [209]:
!cd testsfolder
!pytest -s -k pytest19

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 48 items / 46 deselected / 2 selected                                

testsfolder/pytest19_test.py requested apple
<Function test_request>
None
<Function test_request>
.requested apple
<Function test_request_in_class>
<class 'pytest19_test.TestMyStuff'>
<Function test_request_in_class>
.

======================= 2 passed, 46 deselected in 0.02s =======================


### Parameterizing Fixtures

A single fixture function can be used to define __multiple values__ based on branching by params.

This is seprate from parameterizing tests (which will be covered later).

This relies on using the built-in `request` fixture as shown below.

The flow is as follows:
1. a fixture is marked with a list of parameters in the `params` options
1. the fixture code checks `request.param` to see which param has been passed in from the list
1. the test (or fixture) using it just assumes it could be from any of the params
1. `pytest` will execute __all the permutations of params__
1. `request.node` will be pointing to a parameterized version of the test node (eg. `test_fruit[banana]`)

Note that in the below example, there is only 1 test, but it uses a parameterized fixture with 2 params, so the test gets used twice.  It passes once, and fails once.

In [212]:
%%writefile testsfolder/pytest20_test.py

import pytest

@pytest.fixture(params=['apple', 'banana'])
def my_fruit(request):
    print(request.node)
    requested_fruit = request.param
    if requested_fruit == 'apple':
        return 'apple'
    elif requested_fruit == 'banana':
        return 'banana'
    else:
        raise ValueError

def test_fruit(my_fruit):
    assert my_fruit == 'apple'

Overwriting testsfolder/pytest20_test.py


In [214]:
!cd testsfolder
!pytest -s -k pytest20

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 50 items / 48 deselected / 2 selected                                

testsfolder/pytest20_test.py <Function test_fruit[apple]>
.<Function test_fruit[banana]>
F

=================================== FAILURES ===================================
______________________________ test_fruit[banana] ______________________________

my_fruit = 'banana'

    def test_fruit(my_fruit):
>       assert my_fruit == 'apple'
E       AssertionError: assert 'banana' == 'apple'
E         
E         - apple
E         + banana

testsfolder/pytest20_test.py:16: AssertionError
=========================== short test summary info ============================
FAILED testsfolder/pytest20_test.py::test_fruit[banana] - AssertionError: assert 'banana' == 'apple'
================== 1 failed, 1 p

### Other request.node Members

`request.node` is an instance of `Node`, which you can look up to get the full list.

In [221]:
%%writefile testsfolder/pytest21_test.py

import pytest

@pytest.fixture
def my_fruit(request):
    print('///new request///')
    print('cls:', request.node.cls)
    print('function:', request.node.function)
    print('name:', request.node.name)
    print('parent name:', request.node.parent.name)
    print('path:', request.node.path)
    print('nodeid:', request.node.nodeid)
    return "apple"

def test_request(request, my_fruit):
    assert my_fruit == 'apple'
    
class TestMyStuff:
    def test_request_in_class(self, my_fruit, request):
        pass

Overwriting testsfolder/pytest21_test.py


In [222]:
!cd testsfolder
!pytest -s -k pytest21

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 52 items / 50 deselected / 2 selected                                

testsfolder/pytest21_test.py ///new request///
cls: None
function: <function test_request at 0x1063b2b90>
name: test_request
parent name: pytest21_test.py
path: /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest21_test.py
nodeid: testsfolder/pytest21_test.py::test_request
.///new request///
cls: <class 'pytest21_test.TestMyStuff'>
function: <function TestMyStuff.test_request_in_class at 0x1063b2cb0>
name: test_request_in_class
parent name: TestMyStuff
path: /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest21_test.py
nodeid: testsfolder/pytest21_test.py::TestMyStuff::test_request_in_class
.

======================= 2 passed, 50 deselected in 0.02s =======================

## Markers

Markers are a way of adding __metadata to tests__. They have the following uses:
1. They can be seen via the `request` fixture from within tests and fixtures.
1. They can affect test behavior using built-in markers with special purposes
1. They can be used for filtering tests in the terminal

### Setting and Reading Markers

Markers are essentially __arbitrary names__ on the `pytest.mark` object that you can use as __decorators on tests__.  In our example, we __made up our own__ marker.  There are also several __built-in markers__.

__NOTE__: this example generates __warnings__ because we didn't configure our custom markers (see later).  The tests still work as expected though.

Markers are __set on tests__ only, not on fixtures.  You can set them on __multiple tests__ by setting on a class, etc.

Markers can be __read__ via `request.node` in a test or fixture.  In this example, the `get_closest_marker()` method gets a `Marker` object if the marker exists on the node, or `None` if it does not. This makes the fixture more like a __traditional helper method__ in unit tests.

In [233]:
%%writefile testsfolder/pytest22_test.py

import pytest

@pytest.fixture
def my_fruit(request):
    print(request.node.get_closest_marker('my_marker'))  # check if the marker is applied on the test
    if request.node.get_closest_marker('my_marker'):
        return 'apple'
    else:
        return 'banana'

@pytest.mark.my_marker   # this is a custom marker we made up here
def test_marker(my_fruit):
    assert my_fruit == 'apple'

def test_marker_missing(my_fruit):
    assert my_fruit == 'banana'  # value of fixture depends on presence of the marker
    
@pytest.mark.my_marker # mark the whole class
class TestApples:
    def test_marker(self, my_fruit):
        assert my_fruit == 'apple'
        
    def test_marker2(self, my_fruit):
        assert my_fruit == 'apple'

Overwriting testsfolder/pytest22_test.py


In [234]:
!cd testsfolder
!pytest -s -k pytest22

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 56 items / 52 deselected / 4 selected                                

testsfolder/pytest22_test.py Mark(name='my_marker', args=(), kwargs={})
.None
.Mark(name='my_marker', args=(), kwargs={})
.Mark(name='my_marker', args=(), kwargs={})
.

=============================== warnings summary ===============================
testsfolder/pytest22_test.py:12
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:12: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.my_marker   # this is a custom marker we made up here

testsfolder/pytest22_test.py:19
  /Users/davidpetrofsky/repos/snippets/py

### Custom Marker Warning

The above example used a custom marker, but it got warnings because we didn't configure it properly. To make the warning go away, you should add your marker names (and any params they take) to `pytest.ini`.

To generate an actual __error__ instead of warning, use the `--strict-markers` flag at the terminal.

### Markers with Arguments

You can arbitrarily pass arguments into markers as part of the decorators and read them in a fixture with `Marker.args` on the `request` object after getting a `Marker` object.

`pytest.ini` supports specifying how many args the marker should take.

In [241]:
%%writefile testsfolder/pytest23_test.py

import pytest

@pytest.fixture
def my_fruit(request):
    return request.node.get_closest_marker('my_marker').args[0]

@pytest.mark.my_marker('apple')
def test_marker(my_fruit):
    assert my_fruit == 'apple'

@pytest.mark.my_marker('banana')
def test_marker_2(my_fruit):
    assert my_fruit == 'banana'

Overwriting testsfolder/pytest23_test.py


In [242]:
!cd testsfolder
!pytest -s -k pytest23

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 58 items / 56 deselected / 2 selected                                

testsfolder/pytest23_test.py ..

=============================== warnings summary ===============================
testsfolder/pytest22_test.py:12
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:12: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.my_marker   # this is a custom marker we made up here

testsfolder/pytest22_test.py:19
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:19: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can register custom

### pytestmark

`pytestmark` in a test module can be a __single marker__ or a __list of markers__ to apply to __all tests in the file__.

In [243]:
%%writefile testsfolder/pytest24_test.py

import pytest

pytestmark = pytest.mark.my_marker('apple')

@pytest.fixture
def my_fruit(request):
    return request.node.get_closest_marker('my_marker').args[0]

def test_marker(my_fruit):
    assert my_fruit == 'apple'

def test_marker_2(my_fruit):
    assert my_fruit == 'apple'

Writing testsfolder/pytest24_test.py


In [244]:
!cd testsfolder
!pytest -k pytest24

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 60 items / 58 deselected / 2 selected                                

testsfolder/pytest24_test.py ..                                          [100%]

=============================== warnings summary ===============================
testsfolder/pytest22_test.py:12
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:12: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.my_marker   # this is a custom marker we made up here

testsfolder/pytest22_test.py:19
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:19: PytestUnknownMarkWarning: Unknown pytest.mark.my_m

### Marker Methods in request.node

In [261]:
%%writefile testsfolder/pytest25_test.py

import pytest

@pytest.fixture
def my_fruit(request):
    print('/////NEW REQUEST////')
    print(request.node.nodeid)
    print(request.node.own_markers)  # only markers on the test function/method itself
    print(list(request.node.iter_markers())) # iteration - inheritence included
    print(list(request.node.iter_markers(name='my_class_marker'))) # filter by name
    print(request.node.get_closest_marker('my_class_marker')) # bottom-up search for name
    request.node.add_marker('something')  # adding marker within a fixture/test programatically
    return 'apple'

@pytest.mark.my_function_marker
def test_markers(my_fruit):
    pass

@pytest.mark.my_class_marker
class TestMyStuff:
    def test_markers(self, my_fruit):
        pass
    def test_markers_2(self, my_fruit):
        pass

Overwriting testsfolder/pytest25_test.py


In [262]:
!cd testsfolder
!pytest -s -k pytest25

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 63 items / 60 deselected / 3 selected                                

testsfolder/pytest25_test.py /////NEW REQUEST////
testsfolder/pytest25_test.py::test_markers
[Mark(name='my_function_marker', args=(), kwargs={})]
[Mark(name='my_function_marker', args=(), kwargs={})]
[]
None
./////NEW REQUEST////
testsfolder/pytest25_test.py::TestMyStuff::test_markers
[]
[Mark(name='my_class_marker', args=(), kwargs={})]
[Mark(name='my_class_marker', args=(), kwargs={})]
Mark(name='my_class_marker', args=(), kwargs={})
./////NEW REQUEST////
testsfolder/pytest25_test.py::TestMyStuff::test_markers_2
[]
[Mark(name='my_class_marker', args=(), kwargs={})]
[Mark(name='my_class_marker', args=(), kwargs={})]
Mark(name='my_class_marker', args=(), kwargs={})
.

=========================

### Filtering by Markers

Use the `-m` option to supply a marker - only tests with that marker will be run. You can use __boolean operators__ in a string to combine markers.

You can use the `-v` (verbose) param to see which tests were actually run.

In [265]:
%%writefile testsfolder/pytest26_test.py

import pytest

@pytest.mark.full
class TestEverything:
    def test_thing1(self):
        pass
    def test_thing2(self):
        pass
    
@pytest.mark.smoke
class TestSomeThings:
    def test_thing1(self):
        pass
    def test_thing2(self):
        pass

Overwriting testsfolder/pytest26_test.py


In [267]:
!cd testsfolder
!pytest -v -m smoke -k pytest26

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 67 items / 65 deselected / 2 selected                                

testsfolder/pytest26_test.py::TestSomeThings::test_thing1 PASSED         [ 50%]
testsfolder/pytest26_test.py::TestSomeThings::test_thing2 PASSED         [100%]

=============================== warnings summary ===============================
testsfolder/pytest22_test.py:12
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:12: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.my_marker   # this is a custom marker we made up here

In [268]:
!cd testsfolder
!pytest -v -m "smoke or full" -k pytest26

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 67 items / 63 deselected / 4 selected                                

testsfolder/pytest26_test.py::TestEverything::test_thing1 PASSED         [ 25%]
testsfolder/pytest26_test.py::TestEverything::test_thing2 PASSED         [ 50%]
testsfolder/pytest26_test.py::TestSomeThings::test_thing1 PASSED         [ 75%]
testsfolder/pytest26_test.py::TestSomeThings::test_thing2 PASSED         [100%]

=============================== warnings summary ===============================
testsfolder/pytest22_test.py:12
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:12: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can register custom marks to

### Some Built-In Markers

A big one not covered here is `parameterize`, which will be covered in its own section instead of in markers (since it's that important).

In [287]:
%%writefile testsfolder/pytest27_test.py

import pytest

@pytest.fixture
def my_fruit():
    print('apple requested')
    return 'apple'

@pytest.fixture
def my_fruit2():
    print('banana requested')
    return 'banana'

@pytest.fixture
def my_fruit3():
    print('orange requested')
    return 'orange'

# normal unmarked test
def test_run():
    pass

# skip test(s)
@pytest.mark.skip(reason='I just do not like this test')
def test_skipped():
    pass

# conditional skip
@pytest.mark.skipif(True, reason='I might not like it')
def test_maybe_skipped():
    pass

# expected failure(s)
@pytest.mark.xfail
def test_fail():
    assert False
    
# implicit (unreferenced) fixtures to request
# this is like (autouse) from the other side in a way
@pytest.mark.usefixtures("my_fruit", "my_fruit2")
def test_usefixtures(my_fruit3):
    pass # all 3 get requested by this point

Overwriting testsfolder/pytest27_test.py


In [288]:
!cd testsfolder
!pytest -s -v -k pytest27

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 72 items / 67 deselected / 5 selected                                

testsfolder/pytest27_test.py::test_run PASSED
testsfolder/pytest27_test.py::test_skipped SKIPPED (I just do not li...)
testsfolder/pytest27_test.py::test_maybe_skipped SKIPPED (I might no...)
testsfolder/pytest27_test.py::test_fail XFAIL
testsfolder/pytest27_test.py::test_usefixtures apple requested
banana requested
orange requested
PASSED

=============================== warnings summary ===============================
testsfolder/pytest22_test.py:12
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:12: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can re

## Parameterizing Tests

In the same way that you can duplicate test execution by instantiating the same marker(s) with multiple param(s) implicitly, you can parameterize the test itself (using markers) to duplicate execution with different input values.

The general structure of the args to `parametrize` (watch that spelling) are: `{nameOrTupleOfNames}, [NameOrTuple, NameOrTuple]`. The items in the 2nd param (the list) are specific values for each instantiation of the test function.

In [293]:
%%writefile testsfolder/pytest28_test.py

import pytest

@pytest.mark.parametrize('fruit,count', [('banana', 30), ('apple', 40)])
def test_fruit(fruit, count):
    print(fruit, count)
    
@pytest.mark.parametrize('fruit', ['banana', 'apple'])
def test_fruit2(fruit):
    print(fruit)

Overwriting testsfolder/pytest28_test.py


In [294]:
!cd testsfolder
!pytest -s -k pytest28

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 76 items / 72 deselected / 4 selected                                

testsfolder/pytest28_test.py banana 30
.apple 40
.banana
.apple
.

=============================== warnings summary ===============================
testsfolder/pytest22_test.py:12
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:12: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.my_marker   # this is a custom marker we made up here

testsfolder/pytest22_test.py:19
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:19: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is thi

## Terminal Options

In [317]:
%%writefile testsfolder/pytest29_test.py

import pytest

def test_something():
    print('hi')
    pass

@pytest.mark.smoke
def test_something_else():
    pass

class TestMyStuff:
    def test_something(self):
        pass
    
    def test_something_else(self):
        pass

Overwriting testsfolder/pytest29_test.py


### Default

__autodiscover__ everything in current working directory recursively.

In [298]:
!pytest

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 77 items                                                             

testsfolder/pytest10_test.py .                                           [  1%]
testsfolder/pytest11_test.py ....                                        [  6%]
testsfolder/pytest12_test.py ...                                         [ 10%]
testsfolder/pytest13_test.py ....                                        [ 15%]
testsfolder/pytest14_test.py ..                                          [ 18%]
testsfolder/pytest15_test.py ...                                         [ 22%]
testsfolder/pytest16_test.py ..                                          [ 24%]
testsfolder/pytest18_test.py .                                           [ 25%]
testsfolder/pytest19_test.py ..                                

### Whole Folder

In [324]:
!pytest testsfolder

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 80 items                                                             

testsfolder/pytest10_test.py .                                           [  1%]
testsfolder/pytest11_test.py ....                                        [  6%]
testsfolder/pytest12_test.py ...                                         [ 10%]
testsfolder/pytest13_test.py ....                                        [ 15%]
testsfolder/pytest14_test.py ..                                          [ 17%]
testsfolder/pytest15_test.py ...                                         [ 21%]
testsfolder/pytest16_test.py ..                                          [ 23%]
testsfolder/pytest18_test.py .                                           [ 25%]
testsfolder/pytest19_test.py ..                                

### Specific File

In [318]:
!pytest testsfolder/pytest29_test.py

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 4 items                                                              

testsfolder/pytest29_test.py ....                                        [100%]

=============================== warnings summary ===============================
testsfolder/pytest29_test.py:8
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest29_test.py:8: PytestUnknownMarkWarning: Unknown pytest.mark.smoke - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.smoke

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
========================= 4 passed, 1 warning in 0.01s =========================


### Multiple Specific Files

In [335]:
!pytest testsfolder/pytest29_test.py testsfolder/pytest29_test.py

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 8 items                                                              

testsfolder/pytest29_test.py ........                                    [ 50%]

=============================== warnings summary ===============================
testsfolder/pytest29_test.py:8
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest29_test.py:8: PytestUnknownMarkWarning: Unknown pytest.mark.smoke - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.smoke

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
========================= 8 passed, 1 warning in 0.01s =========================


### Verbose (-v)

Show __which tests__ actually ran.  This __does not imply -s__.

In [319]:
!pytest -v testsfolder/pytest29_test.py

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 4 items                                                              

testsfolder/pytest29_test.py::test_something PASSED                      [ 25%]
testsfolder/pytest29_test.py::test_something_else PASSED                 [ 50%]
testsfolder/pytest29_test.py::TestMyStuff::test_something PASSED         [ 75%]
testsfolder/pytest29_test.py::TestMyStuff::test_something_else PASSED    [100%]

=============================== warnings summary ===============================
testsfolder/pytest29_test.py:8
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest29_test.py:8: PytestUnknownMarkWarning: Unknown pytest.mark.smoke - is this a typo?  You can register custom marks to avoid

### Show Console (-s)

This turns off the __capturing__ for console out and errors so that you can see them without a test having to fail.

In [320]:
!pytest -s testsfolder/pytest29_test.py

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 4 items                                                              

testsfolder/pytest29_test.py hi
....

=============================== warnings summary ===============================
testsfolder/pytest29_test.py:8
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest29_test.py:8: PytestUnknownMarkWarning: Unknown pytest.mark.smoke - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.smoke

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
========================= 4 passed, 1 warning in 0.01s =========================


### Filter By Marker (-m)

Remember that you can also use a __boolean expression__ like `smoke or integration`.

In [321]:
!pytest -v -m smoke testsfolder/pytest29_test.py

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 4 items / 3 deselected / 1 selected                                  

testsfolder/pytest29_test.py::test_something_else PASSED                 [100%]

=============================== warnings summary ===============================
testsfolder/pytest29_test.py:8
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest29_test.py:8: PytestUnknownMarkWarning: Unknown pytest.mark.smoke - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.smoke

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
================== 1 passed, 3 deselected, 1 warning in 0.01

### Specific Function/Method

`nodeid` syntax

In [322]:
!pytest -v testsfolder/pytest29_test.py::test_something
!pytest -v testsfolder/pytest29_test.py::TestMyStuff::test_something

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 1 item                                                               

testsfolder/pytest29_test.py::test_something PASSED                      [100%]

=============================== warnings summary ===============================
testsfolder/pytest29_test.py:8
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest29_test.py:8: PytestUnknownMarkWarning: Unknown pytest.mark.smoke - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.smoke

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
========================= 1 passed, 1 warning in 0.01s =====

### Test Specific Class

`nodeid` syntax

In [323]:
!pytest -v testsfolder/pytest29_test.py::TestMyStuff

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 2 items                                                              

testsfolder/pytest29_test.py::TestMyStuff::test_something PASSED         [ 50%]
testsfolder/pytest29_test.py::TestMyStuff::test_something_else PASSED    [100%]

=============================== warnings summary ===============================
testsfolder/pytest29_test.py:8
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest29_test.py:8: PytestUnknownMarkWarning: Unknown pytest.mark.smoke - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.smoke

-- Docs: https://docs.pytest.org/en/stable/how-to/ca

### Filter by Keyword (-k)

Filters by __substrings__ within __components of the nodeid__.  They can be combined with __boolean logic__ in a string.

You may end up having to specify __multiple similar file paths__ instead if keyword boolean logic is too limited for your case.

In [328]:
!pytest -v testsfolder -k 29

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 80 items / 76 deselected / 4 selected                                

testsfolder/pytest29_test.py::test_something PASSED                      [ 25%]
testsfolder/pytest29_test.py::test_something_else PASSED                 [ 50%]
testsfolder/pytest29_test.py::TestMyStuff::test_something PASSED         [ 75%]
testsfolder/pytest29_test.py::TestMyStuff::test_something_else PASSED    [100%]

=============================== warnings summary ===============================
testsfolder/pytest22_test.py:12
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:12: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can register custom marks to

In [330]:
!pytest -v testsfolder -k 9

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 80 items / 70 deselected / 10 selected                               

testsfolder/pytest19_test.py::test_request PASSED                        [ 10%]
testsfolder/pytest19_test.py::TestMyStuff::test_request_in_class PASSED  [ 20%]
testsfolder/pytest29_test.py::test_something PASSED                      [ 30%]
testsfolder/pytest29_test.py::test_something_else PASSED                 [ 40%]
testsfolder/pytest29_test.py::TestMyStuff::test_something PASSED         [ 50%]
testsfolder/pytest29_test.py::TestMyStuff::test_something_else PASSED    [ 60%]
testsfolder/pytest9_test.py::TestMyStuff::test_1 PASSED                  [ 70%]
testsfolder/pytest9_test.py::TestMyStuff::test_2 PASSED     

In [332]:
!pytest -v testsfolder -k MyStuff

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 80 items / 57 deselected / 23 selected                               

testsfolder/pytest11_test.py::TestMyStuff::test_autouse PASSED           [  4%]
testsfolder/pytest11_test.py::TestMyStuff::test_autouse2 PASSED          [  8%]
testsfolder/pytest11_test.py::TestMyStuff::test_autouse3 PASSED          [ 13%]
testsfolder/pytest13_test.py::TestMyStuff::test_1 PASSED                 [ 17%]
testsfolder/pytest13_test.py::TestMyStuff::test_2 PASSED                 [ 21%]
testsfolder/pytest15_test.py::TestMyStuff::test_1 PASSED                 [ 26%]
testsfolder/pytest15_test.py::TestMyStuff::test_2 PASSED                 [ 30%]
testsfolder/pytest16_test.py::TestMyStuff::test_1 PASSED    

In [333]:
!pytest -v testsfolder -k TestMyStuff::test

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 80 items / 80 deselected / 0 selected                                

=============================== warnings summary ===============================
testsfolder/pytest22_test.py:12
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:12: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.my_marker   # this is a custom marker we made up here

testsfolder/pytest22_test.py:19
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:19: PytestUnknownMarkWarning: Unknown pytest.mark.my

In [334]:
!pytest -v testsfolder -k 'TestMyStuff and test_something'

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 80 items / 78 deselected / 2 selected                                

testsfolder/pytest29_test.py::TestMyStuff::test_something PASSED         [ 50%]
testsfolder/pytest29_test.py::TestMyStuff::test_something_else PASSED    [100%]

=============================== warnings summary ===============================
testsfolder/pytest22_test.py:12
  /Users/davidpetrofsky/repos/snippets/python/testsfolder/pytest22_test.py:12: PytestUnknownMarkWarning: Unknown pytest.mark.my_marker - is this a typo?  You can register custom marks to avoid this warning - for details, see https://docs.pytest.org/en/stable/how-to/mark.html
    @pytest.mark.my_marker   # this is a custom marker we made up here

### Suppressing Warnings (--disable-warnings)

eg. no more warnings about __custom markers__.

In [336]:
!pytest -v testsfolder --disable-warnings -k 29

============================= test session starts ==============================
platform darwin -- Python 3.10.8, pytest-8.4.2, pluggy-1.6.0 -- /Users/davidpetrofsky/miniforge3/envs/ai/bin/python3.10
cachedir: .pytest_cache
rootdir: /Users/davidpetrofsky/repos/snippets/python
plugins: anyio-3.6.2
collected 80 items / 76 deselected / 4 selected                                

testsfolder/pytest29_test.py::test_something PASSED                      [ 25%]
testsfolder/pytest29_test.py::test_something_else PASSED                 [ 50%]
testsfolder/pytest29_test.py::TestMyStuff::test_something PASSED         [ 75%]
testsfolder/pytest29_test.py::TestMyStuff::test_something_else PASSED    [100%]

================ 4 passed, 76 deselected, 10 warnings in 0.03s =================


## pytest.ini

`pytest.ini` in a folder lets you configure tests in that folder and its subfolders.  You can do things like configure default terminal options, define custom markers, etc.

## Plugins

`pytest` supports plugins, and many useful ones exist.

For instance, `pytest-django` (which you `pip install`) lets you run tests within a django project folder (`unittest` based) with some configuration required in `pytest.ini`.

## Hooks

You can use __certain function names__ in a test file to hook into the system.  For example:
- `pytest_configure`
   - after options and config are parsed but before tests are loaded
   - eg. set up __custom markers__ programatically (instead of in ini), configure __logging__
- `pytest_fixture_*`
   - fixture-related steps
- `pytest_runtest_*`
   - setup, teardown, etc.
- `pytest_collection_modifyitems`
   - after set of tests collected and before any tests run
   - eg. so you can add markers before marker filtering happens
     - eg. based on test name so that names can control when run
     
The `request` fixture also has some hook-like things such as `addfinalizer()` to do cleanup (instead of the normal `yield` way).